In [ ]:
import os
import pandas as pd
from icrawler.builtin import GoogleImageCrawler

df = pd.read_csv('recipes_data.csv')
dish_names = df['title'].dropna().unique()

output_dir = 'recipe_images'
os.makedirs(output_dir, exist_ok=True)

downloaded = set()
for fname in os.listdir(output_dir):
    if fname == 'temp':
        continue
    downloaded.add(fname)

for dish in dish_names:
    dest_filename = f"{dish}.jpg"
    if any(dest_filename in existing for existing in downloaded):
        print(f"Skipping {dish}, image already downloaded.")
        continue

    query = f"{dish} recipe"
    temp_dir = os.path.join(output_dir, 'temp')
    os.makedirs(temp_dir, exist_ok=True)
    
    crawler = GoogleImageCrawler(storage={'root_dir': temp_dir})
    try:
        crawler.crawl(keyword=query, max_num=1)
        temp_files = os.listdir(temp_dir)
        if temp_files:
            src_file = os.path.join(temp_dir, temp_files[0])
            final_dest_file = os.path.join(output_dir, dest_filename)
            if os.path.exists(final_dest_file):
                base, ext = os.path.splitext(dest_filename)
                counter = 1
                while os.path.exists(os.path.join(output_dir, f"{base} ({counter}){ext}")):
                    counter += 1
                final_dest_file = os.path.join(output_dir, f"{base} ({counter}){ext}")
            os.rename(src_file, final_dest_file)
            downloaded.add(os.path.basename(final_dest_file))
            print(f"Downloaded image for '{dish}'.")
    except Exception as e:
        print(f"Error downloading image for '{dish}': {e}")
    
    for f in os.listdir(temp_dir):
        os.remove(os.path.join(temp_dir, f))
